# Overview 

In this half of the lecture we will introduce software to display fMRI data on a 3D representation of the cortical surface.

# Goals
* Understand the way pycortex represents a volumetric data set
* Create pycortex Volume objects from 3D data, masked data, and time series data
* Display data on the cortical surface

## Alignment
fMRI measures activity across the whole brain (or large swathes of the brain), but it is difficult to map that activity to specific anatomical locations in the brain based on functional data alone. Many factors make anatomy ambiguous in functional data: **(1)** functional data are low resolution; **(2)** functional slices are not always aligned with the head in a consistent or precise way across subjects; and **(3)** there is considerable variation in anatomy (i.e., specific locations of sulci & gyri) across subjects. Thus, a common step in fMRI processing is to align the functional data with a high-resolution anatomical scan of the same subject. 

## image of alignment (TO COME)

## Image of pycortex aligner


## Cortical surface extraction

fMRI studies often focus on the cerebral cortex (the outermost layer of the brain). Consequently, it is common to display the results of statistical analyses of fMRI data on inflated and flattened representations of the cerebral cortex. Such cortical surface maps provide a way to examine all cortical fMRI data at once, with the anatomical location of the functional data made clear. 

The cortical surface must be computationally extracted from high spatial resolution anatomical MRI scans, and often manually edited (*NOTE: Manual editing to create a good corical surface can take days or weeks of effort! This data is not free!*)

<img src="figures/MPRAGE.png" align='left' style="height: 200px;">

<img src="figures/MPRAGE_wcortex.png" align='left' style="height: 200px;">

<img src="figures/cortex_3views.png" align='left' style="height: 200px;">




In [ ]:
# Load some necessary libraries
import nibabel  # Neuroimaging library to work with MR images
import cortex  # This is our mapping software pycortex
import neurods
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['image.interpolation'] = 'nearest'
matplotlib.rcParams['image.aspect'] = 'auto'
matplotlib.rcParams['image.cmap'] = 'viridis'

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Load data

Same as previously: load using nibabel, use get_data() method of the nibabel data object, transpose resulting data array, and zscore the data. For now, we will treat this as a generic data set (next week, we will learn more about the experiment that generated this data as we begin to actually analyze the data).

In [ ]:
fname = '/home/shared/cogneuro-connector/data/fmri/categories/sub01_categories1_1.nii.gz'
nii = nibabel.load(fname) 
voldata = nii.get_data()
mask_fname = '/home/shared/cogneuro-connector/data/fmri/categories/s01_category_mask_cortical.npz'
mask = np.load(mask_fname)['mask']
data = voldata.T[:, mask]
data = zscore(data, axis=0)

In [ ]:
first_volume = voldata[0]
print("Dimensions of the first volume: {0}".format(first_volume.shape))
print("Dimensions of the data: {0}".format(data.shape))

That is: (time, Z, Y, X). 

# Onward to 3D data visualizations! 

We will use a python module called pycortex to show data in 3D on the brain. This module was developed here at UC Berkeley in the Gallant lab, mostly by James Gao, with help from Alex Huth, Mark Lescroart, and other lab members. The code is freely available online [here](https://github.com/gallantlab/pycortex), and a paper summarizing the code can be found [here](http://journal.frontiersin.org/article/10.3389/fninf.2015.00023/full). 

To map the functional data onto the cortex, pycortex requires at least two things:

1. The cortical surface of the subject. 
    * pycortex stores cortical surface files (and several other files) for each subject in a reliably structured directory of files. Because of this reliable directory structure, all we need to provide to the code is a subject ID string, and the code will be able to find and load the relevant cortical surface files. 
2. The functional to anatomical aligmnent of this data to that cortical surface
    * Alignment of functional data to anatomical data proceeds by an *affine transform*. How this transformation works is beyond the scope of this class, but you can look it up on [wikipedia](https://en.wikipedia.org/wiki/Affine_transformation) or in your favorite linear algebra textbook if you're curious. **[would be good to have some better references here]**. The practical upshot is that a 4x4 matrix of numbers is sufficient to store the 3 rotations (around the x, y, and z axes) and 3 the transformations (in the x, y and z dimensions) that will transform the functional data in space such that they are aligned with the anatomical data (with the cortical surface). In the pycortex code, "transform" is abbreviated in variable names as `xfm`. Just as with the cortical surface, we only need to specify a name for a transform, and the code will know where to find the file that contains the affine transformation matrix. 

In [ ]:
# (1) subject (specifies the cortical surface of the brain)
subject = 'S2' 
# (2) transform = functional-to-anatomical alignment
transform = 'S2_category_auto' 

To display data in pycortex, we need to create a pycortex object that contains all the relevant information: the data, the cortical surface (i.e., the subject) and the affine transform. That object is called a Volume.

In [ ]:
# Create a volume
data_volume = cortex.Volume(first_volume, subject, transform) 

In [ ]:
data_volume

In [ ]:
# Show the volume in a 3D brain (ooooh)
cortex.webgl.show(data_volume)

(Play with buttons: flatten, color map, color limits, depth, )

Relate this back to MNE: this is like a `raw` object in MNE. It's a wrapper around a channels x time array, which allows you to show the data via other functions. This is a common thing to do in compuational work: to create objects that contain some meta-information, and allow you to do interesting things you couldn't do with plain arrays. 

In [ ]:
# Show a flamtap
# (If you get warnings about a module called shapely, ignore them; they are not important.)
_ = cortex.quickflat.make_figure(data_volume)

### TEACHER INFO
*Instructor notes: Play with colormap, color limits, etc*

Note that vmin, vmax are set in creation of VOLUME object; OR fix make_figure() to take vmin, vmax and re-set data volume limits using those values.

## Masks
pycortex can be used to generate cortical masks; this is where the mask we used above came from.

In [ ]:
mask = cortex.db.get_mask(subject, transform, type='thin')
first_volume_masked = voldata[0, mask]
print('Shape of masked data: ', first_volume_masked.shape)

pycortex can also directly visualize masked data

In [ ]:
# Show creation of volume from masked data
kwargs = dict(vmin=-3, vmax=3)
vol = cortex.Volume(data[0], subject, transform, mask=mask, **kwargs)
print(vol)
print(vol.shape)

# ANSWER CELL

*Instructor notes: Note artifacts near beginning / end of scan (visible as high resopnses in movies for first / last few volumes). This has to do with detrending / magnet start-up... Many analyses will simply clip start / end, but for this data they have not been clipped. Ask what this might do to conclusions drawn from data...*

In [ ]:
data.shape

In [ ]:
# Create a movie volume
data_movie = cortex.Volume(data, subject, transform, mask=mask, cmap='RdBu_r', vmin=-3, vmax=3)

In [ ]:
# 3D web display takes a 4D volume, too
cortex.webgl.show(data_movie) # stim='/path/to/localizer_movie.ogg')